In [1]:
# Merging Tables for data feeding

In [2]:
##Merging the tables to match the time stamps
new_train = pd.merge_asof( measurement_case_1['Park Power [KW]'], nwp_case_1, on="Date_Time")
len_of_power = len(measurement_case_1['Park Power [KW]']) 
len_of_train = len(nwp_case_1 ) 
print( len( new_train ))
print( len_of_power )
print( len_of_train )
temp = 35724
new_train = new_train.iloc[ 10 : , :]
new_train = new_train.reset_index()
print( new_train )

NameError: name 'pd' is not defined

In [ ]:
# FFNN 

In [ ]:
import torch
#print(torch.__version__)
#print(torch.version.cuda)
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F

# Load functions
import torch.nn.init as init
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

In [ ]:
# Definition of class for batch generation
from torch.utils.data import Dataset, DataLoader
class TimeSeriesDataSet(Dataset):
  """
  This is a custom dataset class. It can get more complex than this, but simplified so you can understand what's happening here without
  getting bogged down by the preprocessing
  """
  def __init__(self, X, Y, Z):
    self.X = X
    self.Y = Y
    self.Z = Z
    if len(self.X) != len(self.Y) :
      raise Exception("The length of X does not match the length of Y")
    if len(self.X) != len(self.Z) :
      raise Exception("The length of X does not match the length of Z")
    if len(self.Z) != len(self.Y) :
      raise Exception("The length of Z does not match the length of Y")

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    # note that this isn't randomly selecting. It's a simple get a single item that represents an x and y
    _x = self.X[index]
    _y = self.Y[index]
    _z = self.Z[index]

    return _x , _y , _z 

In [ ]:
# Definition of the FFNN architecture
class Net(nn.Module):

    def __init__(self, in_channels, out_channels, learn_bias=False ):
        super(Net, self).__init__()
        
        self.ffn1 = nn.Linear(in_features =in_channels,
                              out_features =out_channels,
                              bias = learn_bias)
        
        self.ffn2 = nn.Linear(in_features =mid_channels,
                              out_features =out_channels,
                              bias = learn_bias)
        
    def forward(self, x):
        
        x= ( self.ffn1(x))
        return x  



In [ ]:
######### not implemented so far=========
def accuracy(ys, ts):
    predictions = torch.max(ys, 1)[1]
    correct_prediction = torch.eq(predictions, ts)
    return torch.mean(correct_prediction.float())

In [ ]:
# Putting to CUDA the dataset
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()




In [ ]:
# Network creation and paramter modification
LEARNING_RATE = 0.001
criterion =  nn.MSELoss()       
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

in_channels_setting = 16
out_channels_setting = 16

batch_size = 1

net = Net(in_channels_setting, out_channels_setting )
# if use_cuda:
#     net.cuda()
#     print('run on cuda')
# print(net)

#deleting the the extra data that doesnt fit the batch size
modulus = len(new_train) %batch_size
new_train_temp = new_train.iloc[ modulus: , :]
new_train_temp = new_train_temp.reset_index()
#print( new_train_temp )

In [ ]:
### Training

In [ ]:
###### setting hyperparameters and gettings epoch sizes
num_epochs = 2
num_samples_train = len( new_train )
num_batches_train = (num_samples_train // batch_size ) 
num_samples_valid = len( new_train )
num_batches_valid = (num_samples_valid // batch_size ) 

# setting up lists for handling loss/accuracy
train_acc, train_loss = [], []
valid_acc, valid_loss = [], []
test_acc, test_loss = [], []

losses_train_train = []
Losses_train_valid = []
Losses_test = []
#get_slice = lambda i, size: range(i * size, (i + 1) * size)

for epoch in range(num_epochs):
    # Forward -> Backprob -> Update params
    ## Train
    train_loader = iter(DataLoader(TimeSeriesDataSet(new_train_temp['Direction_50m'], new_train_temp['Speed_50m'], new_train_temp['Speed_50m'] ), batch_size= batch_size_own, shuffle=False))

    train_train_cur_loss = 0
    net.train()
    print( f'new epoch, number{epoch}')
    for i in range(num_batches_train-10):
        #print( f'{i} run')
        optimizer.zero_grad()
        x, y, z = train_loader.next()
        output = net( y.float() )
        
        
        # compute gradients given loss
        # get the target ==> Z
        batch_loss = criterion(output.float(), z.float())
        batch_loss.backward()
        optimizer.step()

        
        train_train_cur_loss += batch_loss   
    losses_train_train.append(train_train_cur_loss / batch_size)
    
    net.eval()
    ### Evaluate training
    train_loader = iter(DataLoader(TimeSeriesDataSet(new_train_temp['Direction_50m'], new_train_temp['Speed_50m'], new_train_temp['Speed_50m'] ), batch_size= batch_size_own, shuffle=False))
    train_val_cur_loss = 0
    for i in range(num_batches_train):
        x, y, z = train_loader.next()
        output = net( y.float() )
        
        batch_loss = criterion(output.float(), z.float())
        
        train_val_cur_loss += batch_loss.detach().numpy()
    Losses_train_valid.append(train_val_cur_loss / batch_size)
    
    ### Evaluate validation
    test_loader = iter(DataLoader(TimeSeriesDataSet(new_train_temp['Direction_50m'], new_train_temp['Speed_50m'], new_train_temp['Speed_50m'] ), batch_size= batch_size_own, shuffle=False))
    test_cur_loss = 0
    for i in range(num_batches_valid):
        x, y, z = test_loader.next()
        output = net( y.float() )
        
        batch_loss = criterion(output.float(), z.float())
        
        test_cur_loss += batch_loss.detach().numpy()
    Losses_test.append(test_cur_loss / batch_size)
    
    
epoch = np.arange(len(Losses_test))
plt.figure()
plt.plot( Losses_train_valid, 'r', Losses_test, 'b') #, epoch, Losses_test, 'b')
plt.legend(['Train Loss','Validation Loss'])
plt.xlabel('Updates'), plt.ylabel('Acc')